In [1]:
import pandas as pd

In [3]:
data = pd.read_excel("..//OneDrive - UKRI//GGIS_return_23_24_awards_signed_off_13122024.xlsx", 
              sheet_name='Awards to publish')

In [4]:
data['Recipient Category'].unique()

array(['UK Company', 'UK educational institution', 'UK charity', 'Other',
       'International organisation', 'UK public body', 'Charity',
       'UK Public body', 'UK Company '], dtype=object)

In [6]:
data['AllocationMethod'].unique()

array(['General Grants - Competed'], dtype=object)

In [7]:
data_select = data.copy()[['OrganisationAward Reference', 'Award Start Year', 'Award End Year', 'Funding agreement value', 'Recipient Name', 'Recipient Category']]

In [8]:
data_select['Number of years'] = data_select['Award End Year'] - data_select['Award Start Year'] + 1

In [9]:
data_select['Value per year'] = data_select['Funding agreement value']/ data_select['Number of years']

In [10]:
def list_years(row):
    return list(range(row['Award Start Year'], row['Award End Year'] + 1))

In [11]:
data_select['years_active'] = data_select.apply(list_years, axis=1)

In [12]:
data_select

,OrganisationAward Reference,Award Start Year,Award End Year,Funding agreement value,Recipient Name,Recipient Category,Number of years,Value per year,years_active
0,10084492,2023,2025,302604.00,&Sisters Ltd,UK Company,3,100868.00,"[2023, 2024, 2025]"
1,10044614,2022,2023,13619.00,100Daysofnocode Ltd,UK Company,2,6809.50,"[2022, 2023]"
2,10037138,2023,2024,92253.64,10Be5 Ltd.,UK Company,2,46126.82,"[2023, 2024]"
3,10077118,2023,2024,24640.50,1749 Ltd.,UK Company,2,12320.25,"[2023, 2024]"
4,10034038,2022,2024,200010.00,17Cicada Ltd,UK Company,3,66670.00,"[2022, 2023, 2024]"
...,...,...,...,...,...,...,...,...,...
27690,10043198,2022,2023,45464.00,Zucy Ltd,UK Company,2,22732.00,"[2022, 2023]"
27691,10088379,2023,2024,39999.00,Zugbox Limited,UK Company,2,19999.50,"[2023, 2024]"
27692,10043951,2022,2023,42662.00,Zuntold Ltd,UK Company,2,21331.00,"[2022, 2023]"
27693,10074233,2023,2023,49261.00,Zutrex Limited,UK Company,1,49261.00,[2023]


In [13]:
df_explode = data_select.explode(column='years_active')


In [14]:
data_company = df_explode.copy()[(data['Recipient Category'] == 'UK Company') & (~df_explode['Recipient Name'].str.contains('Catapult'))]

In [18]:
data1 = data_company.groupby('years_active', as_index=False).agg({'OrganisationAward Reference':'nunique', 'Value per year':'sum', 'Recipient Name':'nunique'})

In [19]:
data_total = df_explode.groupby('years_active', as_index=False).agg({'OrganisationAward Reference':'nunique', 'Value per year':'sum', 'Recipient Name':'nunique'})

In [28]:
data_out = pd.merge(data1, data_total, on='years_active').drop(columns=['OrganisationAward Reference_y', 'Recipient Name_y'])

data_out.rename(columns={'years_active':'Year','OrganisationAward Reference_x':'Number of awards', 'Recipient Name_x':'Number of orgs', 'Value per year_x':'Value per year'}, inplace=True)

data_out['Value % Total'] = data_out['Value per year']/data_out['Value per year_y']*100

data_out.drop(columns='Value per year_y').to_clipboard(index=False)

In [ ]:
data_out = df_explode.groupby('years_active', as_index=False).agg({'OrganisationAward Reference':'nunique', 'Value per year':'sum', 'Recipient Name':'nunique'})
data_out.rename(columns={'years_active':'Year','OrganisationAward Reference':'Number of awards', 'Recipient Name':'Number of orgs'}).to_clipboard(index=False)



In [48]:
data_group = df_explode.groupby(['Recipient Name', 'years_active'], as_index=False).agg({'Value per year':'sum', 'OrganisationAward Reference':'nunique'})



In [50]:
data_out = data_group.sort_values('Value per year', ascending=False).rename(columns={'years_active':'Year', 'OrganisationAward Reference':'Number of awards'})


In [ ]:
data_out[~data_out['Recipient Name'].str.contains('Catapult')].groupby('Year')

In [ ]:
df_explode[df_explode['Recipient Name'].str.contains('Rolls-Royce')]

In [39]:
data_company.sort_values(by='Funding agreement value', ascending=False).to_clipboard(index=False)

In [36]:
data_company[data_company['Recipient Name'] == 'The Manufacturing Technology Centre Limited'].groupby('years_active').sum(numeric_only=True)

,Award Start Year,Award End Year,Funding agreement value,Number of years,Value per year
years_active,,,,,
2018,4036,4047,9.276814e+07,13,1.542253e+07
2019,4036,4047,9.276814e+07,13,1.542253e+07
2020,10096,10119,9.623771e+07,28,1.607541e+07
2021,18180,18217,1.060068e+08,46,1.824814e+07
2022,48510,48580,1.117282e+08,94,1.971191e+07
2023,88970,89084,2.130493e+08,158,3.887275e+07
2024,80891,80996,1.219259e+08,145,2.358348e+07
2025,44497,44564,1.092546e+08,89,1.953381e+07
2026,20227,20264,1.005190e+08,47,1.744713e+07


In [38]:
39/1027*100


3.79746835443038

In [41]:
203997656.7/1027821950

0.19847567635620156

In [44]:
data[data['Recipient Name'].str.contains('Catapult')]['Recipient Name'].unique()

array(['Cell Therapy Catapult Limited',
       'Compound Semiconductor Applications Catapult Limited',
       'Connected Places Catapult', 'Digital Catapult',
       'Energy Systems Catapult Limited',
       'High Value Manufacturing Catapult',
       'Medicines Discovery Catapult Limited',
       'Offshore Renewable Energy Catapult',
       'Satellite Applications Catapult Limited',
       'Satellite Applications Catapult Services Limited'], dtype=object)

In [46]:
df_explode[df_explode['Recipient Name'].str.contains('Catapult')].groupby('years_active').sum(numeric_only=True)

,Award Start Year,Award End Year,Funding agreement value,Number of years,Value per year
years_active,,,,,
2018,16144,16184,6.002408e+08,48,1.000401e+08
2019,20182,20231,6.946432e+08,59,1.188974e+08
2020,34322,34399,8.271066e+08,94,1.411382e+08
2021,58574,58690,8.444226e+08,145,1.447700e+08
2022,151586,151815,8.697285e+08,304,1.516177e+08
2023,274989,275346,1.582119e+09,493,2.752830e+08
2024,218412,218707,8.850026e+08,403,1.550352e+08
2025,145618,145843,8.724086e+08,297,1.511558e+08
2026,38432,38518,6.978361e+08,105,1.167640e+08


In [47]:
275/1000

0.275